# Импорты

In [73]:
from json import loads, dump
from bs4 import BeautifulSoup
from fake_useragent import  UserAgent
from requests import get, post, RequestException

# Константы

In [74]:
RESULT = []

# Получение данных с хабра

In [75]:
PAGES = 5
URL = "https://habr.com/kek/v2/articles/?query=serverless&order=relevance&fl=ru&hl=ru&perPage=20"

for i in range(1, PAGES + 1):
  response = get(f"{URL}&page={i}")
  text = response.content.decode('utf-8')

  if response.status_code != 200:
    print(f"Запрос завершился неудачей\n{text}")
    break

  content = loads(text)
  publications = content["publicationRefs"]

  for article in publications:
    RESULT.append({
      "title": publications[article]["titleHtml"],
      "author": publications[article]["author"]["fullname"],
      "url": "https://habr.com/ru/articles/" + article + "/"
    })

# Получение данных с Elibrary
Сайт не всегда пропускает запросы, поэтому получение работает через раз

In [ ]:
URL = "https://www.elibrary.ru"
SEARCH_URL = f"{URL}/query_results.asp"
data = {
    "where_fulltext": "on",
    "where_name": "on",
    "where_abstract": "on",
    "where_keywords": "on",
    "where_affiliation": "",
    "where_references": "",
    "type_article": "on",
    "type_disser": "on",
    "type_book": "on",
    "type_report": "on",
    "type_conf": "on",
    "type_patent": "on",
    "type_preprint": "on",
    "type_grant": "on",
    "type_dataset": "on",
    "search_freetext": "",
    "search_morph": "on",
    "search_fulltext": "",
    "search_open": "",
    "search_results": "",
    "titles_all": "",
    "authors_all": "",
    "rubrics_all": "",
    "queryboxid": "",
    "itemboxid": "",
    "begin_year": "",
    "end_year": "",
    "issues": "all",
    "orderby": "rank",
    "order": "rev",
    "changed": "1",
    "ftext": "serverless"
}
headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "ru,en;q=0.9,en-GB;q=0.8,en-US;q=0.7",
    "cache-control": "no-cache",
    "content-Type": "application/x-www-form-urlencoded",
    "pragma": "no-cache",
    "priority": "u=0, i",
    "user-agent": str(UserAgent.chrome),
}

try:
    response = post(SEARCH_URL, data = data, headers = headers)
    text = response.content.decode('utf-8')
    if response.status_code != 200:
        print(f"Запрос завершился неудачей\n{text}")
except RequestException as error:
    text = ""
    print(error)

soup = BeautifulSoup(text, "html.parser")
articles = soup.select("#restab tr[id]>td[align='left']>span")

for article in articles:
    RESULT.append({
      "title": article.select_one("a").get_text(),
      "author": article.select_one("font:nth-child(3)").get_text(),
      "url": URL + article.select_one("a").attrs["href"],
    })

<class 'requests.exceptions.ConnectionError'>
('Connection aborted.', ConnectionResetError(10054, 'Удаленный хост принудительно разорвал существующее подключение', None, 10054, None))


# Запись результата на диск

In [77]:
with open("result.json", "w", encoding="utf-8") as f:
  dump(RESULT, f, ensure_ascii=False)